# TPS Jun 2022

If you like my work, please, leave an upvote: it will be really appreciated and it will motivate me in offering more content to the Kaggle community ! :)

**Libraries**

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator
from matplotlib.colors import ListedColormap
import seaborn as sns
from cycler import cycler
from IPython.display import display
import datetime

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import mutual_info_classif

Let's start by reading the data and looking at the first few rows:

In [ ]:
data = pd.read_csv('../input/tabular-playground-series-jun-2022/data.csv')
sample_submission  = pd.read_csv('../input/tabular-playground-series-jun-2022/sample_submission.csv')

In [ ]:
print()
print('data')
display(data.head())


In [ ]:
data.info()

**visulize for missing values**

In [ ]:
import missingno as msno
%matplotlib inline
msno.matrix(data.sample(250))

In [ ]:
display(data.duplicated().sum())

In [ ]:
float_features = [f for f in data.columns if data[f].dtype == 'float64']

# Training histograms
fig, axs = plt.subplots(4, 4, figsize=(16, 16))
for f, ax in zip(float_features, axs.ravel()):
    ax.hist(data[f], density=True, bins=100)
    ax.set_title(f'Train {f}, std={data[f].std():.1f}')
plt.suptitle('Histograms of the float features', y=0.93, fontsize=20)
plt.show()

In [ ]:
data_completed = pd.DataFrame()

**LGBM Regressor**

In [ ]:
from lightgbm import LGBMRegressor

index_list = data.columns.to_list()

for index in index_list[66:81]:
    if data.iloc[:,66:81][index].isnull().any():
        print(True)
        missing_list = list(np.where(data.iloc[:,66:81][index].isnull())[0])
        no_missing_list = list(np.where(data.iloc[:,66:81][index].isnull() == False)[0])
        train = data.iloc[:,66:81].iloc[no_missing_list,]
        test = data.iloc[:,66:81].iloc[missing_list,]
        X = train.drop([index],axis=1)
        y = train[index]
        X_test = test.drop([index],axis=1)
        model = LGBMRegressor(n_estimators=50000,metric='rmse')
        model.fit(X,y)
        y_predict = model.predict(X_test)
        data_all = data.iloc[:,66:81][index]
        data_all.iloc[missing_list,] = y_predict
        
        data_completed = pd.concat([data_completed,data_all],axis=1)
    else:
        data_completed = pd.concat([data_completed,data[index]],axis=1)
        print(False)

In [ ]:
data[['F_4_0','F_4_1','F_4_2','F_4_3','F_4_4','F_4_5','F_4_6','F_4_7','F_4_8','F_4_9','F_4_10','F_4_11','F_4_12','F_4_13','F_4_14']] =  data_completed[['F_4_0','F_4_1','F_4_2','F_4_3','F_4_4','F_4_5','F_4_6','F_4_7','F_4_8','F_4_9','F_4_10','F_4_11','F_4_12','F_4_13','F_4_14']]

In [ ]:
data.fillna(data.mean(),inplace=True)

**Submission**

In [ ]:
for i, row in enumerate(submission.values):
    row_col = row[0]
    imputed_row = row_col.split("-")[0] 
    imputed_col = row_col.split("-")[1] 
    submission.at[i, "value"] = data.iloc[int(imputed_row)][imputed_col]

In [ ]:
submission.to_csv('submission.csv', index=False)

# Thank you for reading!
# Please let me know if you have any questions and I look forward to any suggestions 🙂